In [1]:
from google.colab import drive
drive.mount('/content/drive')
DATA_path      = '/content/drive/MyDrive/bot_resorces'

Mounted at /content/drive


In [2]:
!pip install PyMuPDF faiss-cpu sentence-transformers huggingface_hub scikit-learn streamlit pyngrok
!pip install -q "sentence-transformers>=2.6.0" "torch>=2.2" faiss-cpu pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 822.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [3]:
%%writefile app.py
# RAGChatbot + Streamlit (fixed CPU dtype + precomputed embeddings)

import os
import numpy as np
import faiss
import fitz
import streamlit as st
import torch

from datetime import datetime
from zoneinfo import ZoneInfo
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
from sklearn.metrics.pairwise import cosine_similarity

chicago_tz = ZoneInfo("America/Chicago")


class RAGChatbot:
    def __init__(
        self,
        pdf_folder: str = "/content/drive/MyDrive/bot_resorces",
        chunk_size: int = 500,
        overlap: int = 0,
        top_k: int = 3,
        hf_api_key: str =  # please use a hugging interface key of your own ,
        hf_model: str = "meta-llama/Llama-3.1-8B-Instruct",
    ):
        # --- Configuration ---
        self.pdf_folder = pdf_folder
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.top_k = top_k
        self.hf_api_key = hf_api_key
        self.hf_model = hf_model

        # --- 1) Load & chunk PDFs ---
        texts = self._extract_texts()
        self.chunks = [c for t in texts for c in self._chunk(t)]

        # --- 2) Embed (force CPU + float32) ---
        # Use full model ID and pin to CPU to avoid dtype issues on CPU
        self.embedder = SentenceTransformer(
            "sentence-transformers/all-mpnet-base-v2",
            device="cpu"
        )
        self.embedder.to(torch.device("cpu"))

        # Convert to numpy + float32 for FAISS
        embeddings = self.embedder.encode(
            self.chunks,
            convert_to_numpy=True,
            normalize_embeddings=False,
            show_progress_bar=False,
        ).astype("float32")

        # Keep embeddings for later rerank (no re-encode)
        self.chunk_embs = embeddings

        # --- 3) FAISS index (L2) ---
        dim = embeddings.shape[1] if embeddings.size else 768  # default
        self.index = faiss.IndexFlatL2(dim)
        if embeddings.size > 0:
            self.index.add(embeddings)

        # --- 4) Hugging Face client ---
        self.client = InferenceClient(
            provider="novita",
            api_key=self.hf_api_key,
        )

    # ---------------- Internal helpers ----------------

    def _extract_texts(self):
        """Read all PDFs in folder into plain text."""
        docs = []
        if not os.path.isdir(self.pdf_folder):
            return docs
        for fn in os.listdir(self.pdf_folder):
            if fn.lower().endswith(".pdf"):
                path = os.path.join(self.pdf_folder, fn)
                try:
                    doc = fitz.open(path)
                    text = []
                    for page in doc:
                        text.append(page.get_text())
                    docs.append("".join(text))
                except Exception as e:
                    # Skip unreadable files but don't crash app
                    print(f"[WARN] Failed to read {path}: {e}")
        return docs

    def _chunk(self, text: str):
        """Simple word-based chunking with optional overlap (in words)."""
        words = text.split()
        if self.chunk_size <= 0:
            return
        step = max(1, self.chunk_size - max(0, self.overlap))
        for i in range(0, len(words), step):
            yield " ".join(words[i : i + self.chunk_size])

    def _retrieve(self, query: str):
        """Retrieve top chunks with FAISS, then rerank by cosine using precomputed embs."""
        if len(self.chunks) == 0:
            return []

        # Query embedding (float32 for FAISS)
        q_emb = self.embedder.encode(
            [query],
            convert_to_numpy=True,
            normalize_embeddings=False,
            show_progress_bar=False,
        )[0].astype("float32")

        # FAISS search over available chunks
        k = min(10, len(self.chunks))
        _, ids = self.index.search(np.array([q_emb]), k)
        cand_idx = ids[0].tolist()

        # Cosine rerank with precomputed embeddings (no re-encode)
        qn = q_emb / (np.linalg.norm(q_emb) + 1e-12)
        scores = []
        for i in cand_idx:
            ce = self.chunk_embs[i]
            cn = ce / (np.linalg.norm(ce) + 1e-12)
            scores.append((float(np.dot(qn, cn)), i))

        scores.sort(reverse=True, key=lambda x: x[0])
        top_idxs = [i for _, i in scores[: self.top_k]]
        return [self.chunks[i] for i in top_idxs]

    # ---------------- HF calls ----------------

    def _call_hf(self, context: str, query: str):
        prompt = (
            "Answer the question using the internal document context provided below.\n"
            "=== DOCUMENT CONTEXT START ===\n"
            + context
            + "\n=== DOCUMENT CONTEXT END ===\n"
            f"Question: {query}\n"
            f"(Asked on {datetime.now(chicago_tz).strftime('%Y-%m-%d')})\n"
            "Answer:"
        )
        completion = self.client.chat.completions.create(
            model=self.hf_model,
            messages=[{"role": "user", "content": prompt}],
        )
        # Be robust to different client return shapes
        choice = completion.choices[0]
        msg = getattr(choice, "message", choice)
        if hasattr(msg, "content"):
            return msg.content
        if isinstance(msg, dict):
            return msg.get("content", "")
        return str(msg)

    def _call_hf_structured(self, prompt: str):
        completion = self.client.chat.completions.create(
            model=self.hf_model,
            messages=[{"role": "user", "content": prompt}],
        )
        choice = completion.choices[0]
        msg = getattr(choice, "message", choice)
        if hasattr(msg, "content"):
            return msg.content
        if isinstance(msg, dict):
            return msg.get("content", "")
        return str(msg)

    # ---------------- Public API ----------------

    def answer(self, query: str) -> str:
        """Retrieve + call HF + return text."""
        top_chunks = self._retrieve(query)
        context = "\n\n".join(top_chunks)
        return self._call_hf(context, query)

    def structured_answer(self, promp: str, query: str) -> str:
        """Same as answer(), but lets you prepend your own instruction/summary."""
        top_chunks = self._retrieve(query)
        context = "\n\n".join(top_chunks)

        prompt = (
            "Answer the question using the internal document context provided below.\n"
            "=== DOCUMENT CONTEXT START ===\n"
            f"{context}\n"
            "=== DOCUMENT CONTEXT END ===\n\n"
            f"{promp}\n\n"
            f"Question: {query}\n"
            f"(Asked on {datetime.now(chicago_tz).strftime('%Y-%m-%d')})\n"
            "Answer:"
        )
        return self._call_hf_structured(prompt)

    # ---------------- History helpers ----------------

    def get_last_n_conversations_as_string(self, n=2):
        history = st.session_state.get("history", [])
        total_pairs = len(history) // 2
        n = min(n, total_pairs)
        recent = history[-2 * n :]
        conversation_string = ""
        for speaker, message in recent:
            conversation_string += f"{speaker}: {message}\n"
        return conversation_string.strip()

    def summarize_history(self, n=2):
        the_conv = self.get_last_n_conversations_as_string(n)
        if not the_conv:
            return "(no previous conversation)"
        prompt = f"Previous conversation:\n{the_conv}\n\nSummarize this clearly."
        completion = self.client.chat.completions.create(
            model=self.hf_model,
            messages=[{"role": "user", "content": prompt}],
        )
        choice = completion.choices[0]
        msg = getattr(choice, "message", choice)
        if hasattr(msg, "content"):
            return msg.content.strip()
        if isinstance(msg, dict):
            return msg.get("content", "")
        return str(msg)


# ---------------- Streamlit UI ----------------

rag = RAGChatbot()

st.set_page_config(page_title="RAG Chatbot", page_icon="🤖")
st.title("📚 RAG-Powered Chatbot")

# Initialize session history
if "history" not in st.session_state:
    st.session_state.history = []

# Display chat history
for speaker, text in st.session_state.history:
    if speaker == "You":
        st.markdown(f"**You:** {text}")
    else:
        st.markdown(f"**Bot:** {text}")

st.markdown("---")

# Chat input
with st.form(key="chat_form", clear_on_submit=True):
    user_input = st.text_input("You:", "")
    submit = st.form_submit_button("Send")

# Process input
if submit and user_input:
    summary = "This is the summary of the previous conversation:\n" + rag.summarize_history(2)
    bot_answer = rag.structured_answer(summary, user_input)  # (promp, query)

    st.session_state.history.append(("You", user_input))
    st.session_state.history.append(("Bot", bot_answer))
    st.rerun()  #

# Clear conversation
if st.button("Clear Conversation"):
    st.session_state.history = []

st.markdown("Built with Streamlit and your RAG backend.")


Writing app.py


In [4]:
!pip install pyngrok
from pyngrok import ngrok, conf

!pip install streamlit

!ngrok config add-authtoken  #please add your own authtoken from ngrok
# Kill any previous streamlit processes
!killall streamlit

# Kill any previous ngrok processes
!killall ngrok

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
streamlit: no process found
ngrok: no process found


In [5]:

!ngrok config add-authtoken  #please add your own authtoken from ngrok
# Kill any previous streamlit processes
!killall streamlit

# Kill any previous ngrok processes
!killall ngrok

# Expose via ngrok
public_url = ngrok.connect(addr="8501")
print("Streamlit app:", public_url)

# Start Streamlit app in background
!streamlit run app.py &


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
streamlit: no process found
ngrok: no process found
Streamlit app: NgrokTunnel: "https://56f490228140.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.219.49:8501

2025-08-19 03:48:53.607315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755575333.851701    1566 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755575333.916729    1566 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755575334.406747    1566 computati